# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

Once the above tasks are completed, you should be able to find some geographical patterns about launch sites.

## Import necessary packages

In [1]:
import folium
import wget
import pandas as pd

from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [2]:
from os.path import exists 

if exists('./data/spacex_launch_geo.csv'):
    df = pd.read_csv('./data/spacex_launch_geo.csv')
else :
    url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
    data_file = wget.download(url, './data/')
    df = pd.read_csv(data_file)

Now, you can take a look at what are the coordinates for each site.


In [3]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
lsites = df[['Launch Site', 'Lat', 'Long', 'class']].groupby(['Launch Site'], as_index=False).first()
lsites = lsites[['Launch Site', 'Lat', 'Long']]
print(lsites.shape)
lsites.head()

(4, 3)


,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [4]:
# Start location is NASA Johnson Space Center
nasa_coordinates = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinates, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [5]:
# Create a blue circle at NASA Johnson Space Center, let a popup show its name
circle = folium.Circle(nasa_coordinates, radius=1000, color='#186adb', fill=True)
circle = circle.add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Johnson Space Center with an icon showing its name
marker = folium.map.Marker(
    nasa_coordinates,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#186adb;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


In [6]:
site_map = folium.Map(location=nasa_coordinates, zoom_start=5)

for i in range(lsites.shape[0]):
    site = lsites.iloc[i]
    print(site['Launch Site'], site.Lat, site.Long)
    circle = folium.Circle([site.Lat, site.Long], radius=1000, color='#186adb', fill=True)
    circle = circle.add_child(folium.Popup(site['Launch Site']))

    marker = folium.map.Marker(
        [site.Lat, site.Long],
        icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0),
                     html='<div style="font-size: 12; color:#186adb;"><b>%s</b></div>' % site['Launch Site'],
            ))
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

CCAFS LC-40 28.56230197 -80.57735648
CCAFS SLC-40 28.56319718 -80.57682003
KSC LC-39A 28.57325457 -80.64689529
VAFB SLC-4E 34.63283416 -120.6107455


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [7]:
df.tail()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.0,GTO,SES,Controlled (ocean),0,28.563197,-80.57682
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.0,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.57682
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.0,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.57682
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.0,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.57682
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.0,GTO,SES,No attempt,0,28.563197,-80.57682


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a `MarkerCluster` object

In [8]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [9]:
def assign_marker_color(outcome):
    return 'green' if outcome == 1 else 'red'
    
df['marker_color'] = df['class'].apply(assign_marker_color)
df.tail()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long,marker_color
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.0,GTO,SES,Controlled (ocean),0,28.563197,-80.57682,red
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.0,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.57682,red
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.0,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.57682,red
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.0,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.57682,green
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.0,GTO,SES,No attempt,0,28.563197,-80.57682,red


*TODO:* For each launch result in `df` data frame, add a `folium.Marker` to `marker_cluster`


In [10]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

for index, record in df.iterrows():
    marker = folium.Marker([record.Lat, record.Long],
                           icon=folium.Icon(color='white', icon_color=record.marker_color))
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [11]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',    
    prefix='Lat:',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [12]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [13]:
# Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.
closest_coastline = {'Lat': 28.56289, 'Long': -80.568}
launchsite_row = df.loc[df['Launch Site'] == 'CCAFS SLC-40'].iloc[0, :]
dist_from_coastline = calculate_distance(
    closest_coastline['Lat'], closest_coastline['Long'],
    launchsite_row.Lat, launchsite_row.Long
)
print('The closes coastline is %.2f' % dist_from_coastline, 'km away from site', launchsite_row['Launch Site'])

The closes coastline is 0.86 km away from site CCAFS SLC-40


*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [14]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
distance_marker = folium.Marker(
   [closest_coastline['Lat'], closest_coastline['Long']],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} km".format(dist_from_coastline),
       )
   )

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [15]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates = [[closest_coastline['Lat'], closest_coastline['Long']],[launchsite_row.Lat, launchsite_row.Long]]
line = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(line)
site_map.add_child(distance_marker)

Your updated map with distance line should look like the following screenshot:


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


In [16]:
# Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.
closest_city = {'Lat': 28.6123, 'Long': -80.80786}
launchsite_row = df.loc[df['Launch Site'] == 'KSC LC-39A'].iloc[0, :]
dist_from_city = calculate_distance(
    closest_city['Lat'], closest_city['Long'],
    launchsite_row.Lat, launchsite_row.Long
)
print('The city Titusville is %.2f' % dist_from_city, 'km away from site', launchsite_row['Launch Site'])

The city Titusville is 16.31 km away from site KSC LC-39A


In [17]:
city_dist_marker = folium.Marker(
   [closest_city['Lat'], closest_city['Long']],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} km".format(dist_from_city),
       )
   )

In [18]:
coordinates = [[closest_city['Lat'], closest_city['Long']],[launchsite_row.Lat, launchsite_row.Long]]
line = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(line)
site_map.add_child(city_dist_marker)

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.
